# PCA on wine quality data
We like this data, because it is all numeric data.
It looks like this

```
"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"
7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5
7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5
7.8;0.76;0.04;2.3;0.092;15;54;0.997;3.26;0.65;9.8;5
```

Check data in *datasets/wine-quality*
- winequality-red.csv
- winequality-white.csv

## Step 1 : Load Data

In [ ]:
import os
import urllib.request

## red wine
data_location = '../data/wine-quality/winequality-red.csv'
data_url = 'https://elephantscale-public.s3.amazonaws.com/data/wine-quality/winequality-red.csv'

## white wine
# data_location= '../data/wine-quality/winequality-white.csv'
# data_location =  'https://elephantscale-public.s3.amazonaws.com/data/wine-quality/winequality-white.csv'

if not os.path.exists (data_location):
    data_location = os.path.basename(data_location)
    if not os.path.exists(data_location):
        print("Downloading : ", data_url)
        urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)



In [ ]:
import pandas as pd

dataset = pd.read_csv(data_location, sep=";")
dataset = dataset.dropna()
dataset.sample(10)

## Step 2 : Basic data analysis

In [ ]:
column_to_remove = 'quality'

dataset2 = dataset.drop(column_to_remove, axis=1)

print("original data columns  ", len(dataset.columns))

features = list(dataset2)
print("features: " + str(features))
dataset2

In [ ]:
## basic data analytics
dataset2.describe()

## Step 3 : Create feature vector

In [ ]:
feature_vector = dataset2
feature_vector

## Step 4 : Correlation Matrix of original data
Do see any correlation?

Which columns have the strongest correlation (positive or negative) with
quality?

Write some python code that will find and rank all columns by correlation.

In [ ]:
dataset2.corr()

## TODO: Rank columns by correlation

## Step 5 : Scale Data
We need to scale data before PCA

In [ ]:
feature_vector = (feature_vector - feature_vector.mean()) / feature_vector.std()
feature_vector

## Step 6 : Do PCA

In [ ]:
# numver of principal components 
num_pc = 5

from sklearn.decomposition import PCA

pca = PCA(n_components = num_pc)
pca.fit(feature_vector)
transformed_v = pca.transform(feature_vector)
transformed_v_df = pd.DataFrame(transformed_v, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
transformed_v_df

## Step 7 : Correlation Matrix for Principal Components
These should be very small (close to zero!)

In [ ]:
transformed_v_df.corr().round(3)

## Step 8 : Calculate PC Variance

We started with 5 PCs.  
How much coverage (variance) are we getting?

Play with **num_pc** in Step-6 to get 90% coverage


In [ ]:
import numpy as np

## variance
variance = pca.explained_variance_ratio_
print(variance)
print ("Original data had {} features,  principal components {}".format(len(dataset2.columns), num_pc))
print("Cumulative Explained Variance: " + str(np.cumsum(variance)[-1]))

## Step 9 : Screeplot
Screeplot goes from 0.0  to 1.0

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,5))
sing_vals = np.arange(num_pc) + 1
plt.plot(np.arange(num_pc) + 1, np.cumsum(variance), 'ro-', linewidth=2)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance')


leg = plt.legend(['Explained Variance'], loc='best', borderpad=0.3, 
                 shadow=False, prop=matplotlib.font_manager.FontProperties(size='small'),
                 markerscale=0.4)

In [ ]:

for x in range(0,5):
    # Let's get the top components of PC1:
    print("top components of PC" + str(x+1) + ":")
    rel_values = np.abs(pca.components_[x])/np.sum(np.abs(pca.components_[x]))
    print("Feature Names: " + str([features[i] for i in np.argsort(-rel_values)[:3]]))
    print("Percentages: " + str(rel_values[np.argsort(-rel_values)[:3]]))
    print()

## Step 10 : Biplot

Let's reduce dimensions down to 2 dimensions, and then we can do our biplot.  A biplot plots 2 PCA'ed dimensions, and then also projects the original feature vector onto those two axes.  This helps us see and visualize how the principal components are related to the original features.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def biplot(score,coeff,y,labels=None):
    plt.rcParams['figure.figsize'] = [15, 10]
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.scatter(xs * scalex,ys * scaley, c = y)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
    plt.xlim(-0.6,0.8)
    plt.ylim(-0.6,0.8)
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))
    plt.grid()

In [ ]:
from sklearn.decomposition import PCA

pca_2d = PCA(n_components = 2)

x_new = pca_2d.fit_transform(feature_vector)


# Let's do a biplot of a PCA = 2 dimensions
biplot(x_new[:,0:2],np.transpose(pca.components_[0:2, :]),dataset['quality'],labels=features)


### Interpret biplot

What does the biplot tell us?  Of the two principal components, which of the original features is strongly captured in PC1?  What about PC2?